In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
from scipy.stats import bootstrap
from statsmodels.stats import proportion

import pysindy as ps

# Import Machine Scientist ODE
from importlib.machinery import SourceFileLoader

# Get the absolute path of the script's directory
script_dir = os.path.dirname(os.path.abspath("__file__"))
# Define the relative path to the module
relative_module_path = "rguimera-machine-scientist/machinescientist_ode.py"
path = os.path.join(script_dir, relative_module_path)
ms = SourceFileLoader("ms", path).load_module()

# Import Machine Scientist FIT
from importlib.machinery import SourceFileLoader

# Get the absolute path of the script's directory
script_dir = os.path.dirname(os.path.abspath("__file__"))
# Define the relative path to the module
relative_module_path = "rguimera-machine-scientist/machinescientist_fit.py"
path = os.path.join(script_dir, relative_module_path)
ms_fit = SourceFileLoader("ms_fit", path).load_module()

In [ ]:
# files=[f for f in os.listdir('noise_data/')]
# print(files)
def get_nonzero_terms(model):
    nonzero_indices = np.nonzero(model.coefficients())[
        1
    ]  # Get indices of non-zero terms
    term_names = model.get_feature_names()  # Get feature names
    nonzero_terms = [term_names[i] for i in nonzero_indices]
    return nonzero_terms


def check_sindy_model(model):
    nonzero_terms = get_nonzero_terms(model)
    if nonzero_terms == ["B", "BB"] or nonzero_terms == ["B", "B^2"]:
        return True
    else:
        return False
sigmas = [
    0.005,
    0.006,
    0.007,
    0.008,
    0.009,
    0.01,
    0.02,
    0.03,
    0.04,
    0.05,
    0.06,
    0.07,
    0.08,
    0.09,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
]
error_sampling_bms = []
error_sampling_fit = []
error_sampling_smooth = []
error_sampling_sindy = []
error_sampling_sindymod = []
error_sampling_bms_err = []
error_sampling_fit_err = []
error_sampling_smooth_err = []
error_sampling_sindy_err = []
error_sampling_sindymod_err = []
error_sampling_bms_bern_err = [[], []]
error_sampling_fit_bern_err = [[], []]
error_sampling_smooth_bern_err = [[], []]
error_sampling_sindy_bern_err = [[], []]
error_sampling_sindymod_bern_err = [[], []]
for sigma in sigmas:
    print("#" * 20)
    print(sigma)
    true_bms = []
    true_fit = []
    true_smooth = []
    true_sindy = []
    true_sindymod = []
    count = 0
    for i in range(0, 40):  # [34,38,39]:
        file = f"{sigma}_{i}.pkl"
        # BMS
        try:
            with open(
                f"./noise_data_res_exhaustive_new/BMS_{file[:-4]}.pkl", "rb"
            ) as f:
                # A new file will be created
                bms = pickle.load(f)
            print(str(bms))
            if str(bms).strip() == "((_a0_ * B) + (_a1_ * (B ** 2)))".strip():
                print("BMS True")
                true_bms.append(1.0)
            elif (
                str(bms).strip() == "((_a0_ + (_a1_ * B)) + (_a2_ * (B ** 2)))".strip()
                and bms.par_values["A0"]["_a0_"] < 0.01
            ):
                print(bms.par_values)
                true_bms.append(1.0)
            else:
                print(
                    "BMS False",
                    repr(str(bms)),
                    repr("((_a0_ * B) + (_a1_ * (B ** 2)))"),
                )
                true_bms.append(0.0)
            print(len(bms.x["A0"]["B"].to_numpy()))
            with open(
                f"./noise_data_res_exhaustive_new/BMS_fit_{file[:-4]}.pkl", "rb"
            ) as f:
                # A new file will be created
                bms_fit = pickle.load(f)
            print(str(bms_fit))
            if str(bms_fit).strip() == "((_a0_ * B) + (_a1_ * (B ** 2)))".strip():
                print("FIT True")
                true_fit.append(1.0)
            elif (
                str(bms_fit).strip()
                == "((_a0_ + (_a1_ * B)) + (_a2_ * (B ** 2)))".strip()
                and bms_fit.par_values["A0"]["_a0_"] < 0.01
            ):
                print(bms_fit.par_values)
                true_fit.append(1.0)
            else:
                print(
                    "FIT False",
                    repr(str(bms_fit)),
                    repr("((_a0_ * B) + (_a1_ * (B ** 2)))"),
                )
                true_fit.append(0.0)
            with open(
                f"./noise_data_res_exhaustive_new/BMS_smooth_{file[:-4]}.pkl", "rb"
            ) as f:
                # A new file will be created
                bms_smooth = pickle.load(f)
            print(str(bms_smooth))
            if str(bms_smooth).strip() == "((_a0_ * B) + (_a1_ * (B ** 2)))".strip():
                print("SMOOTH True")
                true_smooth.append(1.0)
            elif (
                str(bms_smooth).strip()
                == "((_a0_ + (_a1_ * B)) + (_a2_ * (B ** 2)))".strip()
                and bms_smooth.par_values["A0"]["_a0_"] < 0.01
            ):
                print(bms_smooth.par_values)
                true_smooth.append(1.0)
            else:
                print(
                    "SMOOTH False",
                    repr(str(bms_smooth)),
                    repr("((_a0_ * B) + (_a1_ * (B ** 2)))"),
                )
                true_smooth.append(0.0)
            # Sindy
            data = pd.read_pickle(
                f"/export/home/oriolca/BMS_ODE/Logistic/noise_data/{file}"
            )

            # sindy_input=[data['x'].values.tolist(),data['y'].values.tolist()]
            print(data)
            X = data["B"].values
            # print(X)
            time = data["t"].values

            ###############################################################################################
            # Weak sindy
            n_models = 1000
            library_functions = [
                lambda x: x,
                lambda x: x * x,
                lambda x: x * x * x,
                lambda x: x * x * x * x,
            ]
            library_function_names = [
                lambda x: x,
                lambda x: x + x,
                lambda x: x + x + x,
                lambda x: x + x + x + x,
            ]
            feature_names = ["B"]
            ensemble_optimizer = ps.STLSQ()

            weak_lib = ps.WeakPDELibrary(
                library_functions=library_functions,
                function_names=library_function_names,
                include_bias=True,
                spatiotemporal_grid=data.t.to_numpy(),
                is_uniform=True,
                K=1000,
            )

            # SINDy initialization
            model = ps.SINDy(
                feature_names=feature_names,
                feature_library=weak_lib,
                optimizer=ensemble_optimizer,
            )
            model.fit(X, ensemble=True, n_models=n_models, quiet=True)
            model.print()
            print(model.coefficients())
            if check_sindy_model(model):
                true_sindy.append(1.0)
                print("Sindy True")
            else:
                true_sindy.append(0.0)
                print("Sindy False")
            #####################################################################################################
            # Regurar sindy
            n_models = 1000
            ensemble_optimizer = ps.STLSQ()
            poly_library = ps.PolynomialLibrary(degree=4, include_bias=True)
            model = ps.SINDy(
                feature_names=["B"],
                feature_library=poly_library,
                optimizer=ensemble_optimizer,
            )
            dt = data.t[1] - data.t[0]
            model.fit(X, t=dt, ensemble=True, n_models=n_models, quiet=True)

            model.print()
            print(model.coefficients())
            if check_sindy_model(model):
                true_sindymod.append(1.0)
                print("Sindy True")
            else:
                true_sindymod.append(0.0)
                print("Sindy False")
            count += 1.0
        except Exception as e:
            print(e)
            pass
    error_sampling_bms.append(np.mean(true_bms))
    error_sampling_bms_err.append(
        bootstrap(
            (true_bms,), np.mean, confidence_level=0.95, method="percentile"
        ).standard_error
    )
    error_sampling_fit.append(np.mean(true_fit))
    error_sampling_fit_err.append(
        bootstrap(
            (true_fit,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )
    error_sampling_smooth.append(np.mean(true_smooth))
    error_sampling_smooth_err.append(
        bootstrap(
            (true_smooth,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )
    error_sampling_sindy.append(np.mean(true_sindy))
    error_sampling_sindy_err.append(
        bootstrap(
            (true_sindy,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )
    error_sampling_sindymod.append(np.mean(true_sindymod))
    error_sampling_sindymod_err.append(
        bootstrap(
            (true_sindymod,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )

####################
0.005
((_a0_ * B) + (_a1_ * (B ** 2)))
BMS True
120
((_a0_ * B) + (_a1_ * (B ** 2)))
FIT True
((_a0_ * B) + (_a1_ * (B ** 2)))
SMOOTH True
            B    t
0    0.015002 -6.0
1   -0.001333 -5.9
2    0.011778 -5.8
3    0.000687 -5.7
4   -0.000684 -5.6
..        ...  ...
115  0.989527  5.5
116  0.998492  5.6
117  1.000608  5.7
118  1.006852  5.8
119  0.991082  5.9

[120 rows x 2 columns]
(B)' = 0.833 B + -1.676 BBB + 0.850 BBBB
[[ 0.          0.83251816  0.         -1.67642673  0.84967324]]
Sindy False
(B)' = 0.980 B + -0.918 B^2 + -0.070 B^3
[[ 0.          0.97965149 -0.91821891 -0.06951828  0.        ]]
Sindy False
((_a0_ * B) + (_a1_ * (B ** 2)))
BMS True
120
((_a0_ * B) + (_a1_ * (B ** 2)))
FIT True
((_a0_ * B) + (_a1_ * (B ** 2)))
SMOOTH True
            B    t
0   -0.006856 -6.0
1    0.012679 -5.9
2    0.005688 -5.8
3    0.009192 -5.7
4   -0.000324 -5.6
..        ...  ...
115  0.998824  5.5
116  1.007034  5.6
117  0.996063  5.7
118  0.993314  5.8
119  0.99344

In [ ]:
data_store = {
    "sigmas": sigmas,
    "error_sampling_bms": error_sampling_bms,
    "error_sampling_fit": error_sampling_fit,
    "error_sampling_smooth": error_sampling_smooth,
    "error_sampling_wsindy": error_sampling_sindy,
    "error_sampling_intsindy": error_sampling_sindymod,
    "error_sampling_bms_err": error_sampling_bms_err,
    "error_sampling_fit_err": error_sampling_fit_err,
    "error_sampling_smooth_err": error_sampling_smooth_err,
    "error_sampling_wsindy_err": error_sampling_sindy_err,
    "error_sampling_intsindy_err": error_sampling_sindymod_err,
}
with open("Plots/detection_sigma.pkl", "wb") as file:
    pickle.dump(data_store, file=file)

In [ ]:
error_sampling_bms = []
error_sampling_fit = []
error_sampling_smooth = []
error_sampling_sindy = []
error_sampling_sindymod = []
error_sampling_bms_err = []
error_sampling_fit_err = []
error_sampling_smooth_err = []
error_sampling_sindy_err = []
error_sampling_sindymod_err = []
error_sampling_bms_bern_err = [[], []]
error_sampling_fit_bern_err = [[], []]
error_sampling_smooth_bern_err = [[], []]
error_sampling_sindy_bern_err = [[], []]
error_sampling_sindymod_bern_err = [[], []]

size = list(range(30, 91, 10)) + list(range(100, 1001, 100))
for length in size:
    print("#" * 20)
    print(length)
    true_bms = []
    true_fit = []
    true_smooth = []
    true_sindy = []
    true_sindymod = []
    count = 0
    for i in range(0, 40):  # [34,38,39]:
        file = f"{i}_{length}.pkl"
        # BMS
        try:
            # BMS ODE
            with open(
                f"./noise_data_res_exhaustive_new/BMS_{file[:-4]}.pkl", "rb"
            ) as f:
                # A new file will be created
                bms = pickle.load(f)
            print(str(bms))
            if str(bms) == "((_a0_ * B) + (_a1_ * (B ** 2)))":
                print("BMS True")
                true_bms.append(1.0)
            else:
                print("BMS False")
                true_bms.append(0.0)
            # BMS Fit
            with open(
                f"./noise_data_res_exhaustive_new/BMS_fit_{file[:-4]}.pkl", "rb"
            ) as f:
                # A new file will be created
                bms_fit = pickle.load(f)
            print(str(bms_fit))
            if str(bms_fit) == "((_a0_ * B) + (_a1_ * (B ** 2)))":
                print("Fit True")
                true_fit.append(1.0)
            else:
                print("Fit False")
                true_fit.append(0.0)
            # BMS Smooth
            with open(
                f"./noise_data_res_exhaustive_new/BMS_smooth_{file[:-4]}.pkl", "rb"
            ) as f:
                # A new file will be created
                bms_smooth = pickle.load(f)
            print(str(bms_smooth))
            if str(bms_smooth) == "((_a0_ * B) + (_a1_ * (B ** 2)))":
                print("Smooth True")
                true_smooth.append(1.0)
            else:
                print("Smooth False")
                true_smooth.append(0.0)
            # Sindy
            data = pd.read_pickle(
                f"/export/home/oriolca/BMS_ODE/Logistic/noise_data_sigma0.05/{file}"
            )
            print(data)
            X = data["B"].values
            # print(X)
            time = data["t"].values
            ###############################################################################################
            # Weak sindy
            n_models = 1000
            library_functions = [
                lambda x: x,
                lambda x: x * x,
                lambda x: x * x * x,
                lambda x: x * x * x * x,
            ]
            library_function_names = [
                lambda x: x,
                lambda x: x + x,
                lambda x: x + x + x,
                lambda x: x + x + x + x,
            ]
            feature_names = ["B"]
            ensemble_optimizer = ps.STLSQ()

            weak_lib = ps.WeakPDELibrary(
                library_functions=library_functions,
                function_names=library_function_names,
                include_bias=True,
                spatiotemporal_grid=data.t.to_numpy(),
                is_uniform=True,
                K=1000,
            )

            # SINDy initialization
            model = ps.SINDy(
                feature_names=feature_names,
                feature_library=weak_lib,
                optimizer=ensemble_optimizer,
            )
            model.fit(X, ensemble=True, n_models=n_models, quiet=True)
            model.print()
            print(model.coefficients())
            if check_sindy_model(model):
                true_sindy.append(1.0)
                print("Sindy True")
            else:
                true_sindy.append(0.0)
                print("Sindy False")
            #####################################################################################################
            # Regurar sindy
            n_models = 1000
            ensemble_optimizer = ps.STLSQ()
            poly_library = ps.PolynomialLibrary(degree=4, include_bias=True)
            model = ps.SINDy(
                feature_names=["B"],
                feature_library=poly_library,
                optimizer=ensemble_optimizer,
            )
            dt = data.t[1] - data.t[0]
            model.fit(X, t=dt, ensemble=True, n_models=n_models, quiet=True)

            model.print()
            print(model.coefficients())
            if check_sindy_model(model):
                true_sindymod.append(1.0)
                print("Sindy True")
            else:
                true_sindymod.append(0.0)
                print("Sindy False")
            count += 1.0
        except Exception as e:
            print(e)
            pass
    error_sampling_bms.append(np.mean(true_bms))
    error_sampling_bms_err.append(
        bootstrap(
            (true_bms,), np.mean, confidence_level=0.95, method="percentile"
        ).standard_error
    )
    error_sampling_fit.append(np.mean(true_fit))
    error_sampling_fit_err.append(
        bootstrap(
            (true_fit,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )
    error_sampling_smooth.append(np.mean(true_smooth))
    error_sampling_smooth_err.append(
        bootstrap(
            (true_smooth,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )
    error_sampling_sindy.append(np.mean(true_sindy))
    error_sampling_sindy_err.append(
        bootstrap(
            (true_sindy,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )
    error_sampling_sindymod.append(np.mean(true_sindymod))
    error_sampling_sindymod_err.append(
        bootstrap(
            (true_sindymod,),
            np.mean,
            confidence_level=0.95,
            random_state=1,
            method="percentile",
        ).standard_error
    )

In [ ]:
data_store = {
    "size": size,
    "error_sampling_bms": error_sampling_bms,
    "error_sampling_fit": error_sampling_fit,
    "error_sampling_smooth": error_sampling_smooth,
    "error_sampling_wsindy": error_sampling_sindy,
    "error_sampling_intsindy": error_sampling_sindymod,
    "error_sampling_bms_err": error_sampling_bms_err,
    "error_sampling_fit_err": error_sampling_fit_err,
    "error_sampling_smooth_err": error_sampling_smooth_err,
    "error_sampling_wsindy_err": error_sampling_sindy_err,
    "error_sampling_intsindy_err": error_sampling_sindymod_err,
}
with open("Plots/detection_length.pkl", "wb") as file:
    pickle.dump(data_store, file=file)